In [1]:
%load_ext autoreload
%autoreload 2

## API endpoints:


I. Endpoints for fetching rules

- Give me all the rules
- Give me rules for the datatypes X,Y,Z
- Give me rules applicable to the datatypes X,Y,Z given input filters

They return:
- a list of rules with their meta-data: id/name/desc (premise/query ?)

---------------------

II. Endpoinds for applying rules

- Apply rule A given input filters
- Apply rules A,B,C given input filters (?)

They return:
- a list of result resource IDs or full payloads (ids passed to ES views by atals?)


ISSUES TO RESOLVE:

- Convention for the filters: how atlas knows which params need to be specified for different rules
- Permission:
    - fetching rules: service account that can always access the project with rules.
    - applying rules: we need to get USER token from atlas and use it when applying the rules. handle access denied in some sessions.
- Service sessions for storing forge sessions for different users.
- Pagination of results.

In [2]:
import getpass

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import fetch_rules, check_premises, apply_rule, get_rule_parameters

In [3]:
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"

In [7]:
TOKEN = getpass.getpass()

In [8]:
RULES_ORG = "bbp"
RULES_PROJECT = "inference-rules"
RULES_VIEW = "https://bbp.epfl.ch/neurosciencegraph/data/rule-view"

In [9]:
rules_forge = KnowledgeGraphForge(
    "../../configs/new-forge-config.yaml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket=f"{RULES_ORG}/{RULES_PROJECT}")

In [10]:
# This could be replaced by any util kg-inference-api implements to handle user forge sessions
FORGE_SESSIONS = {}
def allocate_forge_session(org, project):
    if (org, project) not in FORGE_SESSIONS:
        session = KnowledgeGraphForge(
            "../../configs/new-forge-config.yaml",
            endpoint=ENDPOINT,
            token=TOKEN, 
            bucket=f"{org}/{project}")
        FORGE_SESSIONS[(org, project)] = session
    return FORGE_SESSIONS[(org, project)]

# Fetch rules

1. Fetch all the rules

In [14]:
rules = fetch_rules(rules_forge, RULES_VIEW)
for i, r in enumerate(rules):
    if r["name"] == "Generalise to subClasses":
        rules.pop(i)
        continue
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

Name:  Shape-based morphology recommendation
Desc:  Get closest morphologies according to neurite features and TMD-based vectorization of persistence diagrams
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'type': 'list', 'description': 'List of URIs of similarity models to ignore.', 'name': 'IgnoreModelsParameter', 'optional': True}

Name:  Location-based morphology recommendation
Desc:  Get closest morphologies according to their location given by: coordinates, brain region embedding (ontology-based), axon/dendrite co-projection.
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'type': 'list', 'description': 'List of URIs of sim

2. Fetch rules for a particular data type

In [15]:
morph_rules = fetch_rules(rules_forge, RULES_VIEW, resource_types=["NeuronMorphology"])
for r in morph_rules:
#     print(r)
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

Name:  Shape-based morphology recommendation
Desc:  Get closest morphologies according to neurite features and TMD-based vectorization of persistence diagrams
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'type': 'list', 'description': 'List of URIs of similarity models to ignore.', 'name': 'IgnoreModelsParameter', 'optional': True}

Name:  Location-based morphology recommendation
Desc:  Get closest morphologies according to their location given by: coordinates, brain region embedding (ontology-based), axon/dendrite co-projection.
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'type': 'list', 'description': 'List of URIs of sim

# Check premises of rules (find rules applicable to input filters)

In [16]:
input_filters = {
    "TargetResourceParameter": "https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/85a140b6-8495-4838-9e4f-e347b2a7838f"
}

If premise is satified given the input filters, the rule is applicable.

In [17]:
for rule in morph_rules:
    print(rule["name"])
    print("\t-> Input filters:")
    print("\t\tPremise satisfied: ", check_premises(allocate_forge_session, rule, input_filters))

Shape-based morphology recommendation
	-> Input filters:
		Premise satisfied:  True
Location-based morphology recommendation
	-> Input filters:
		Premise satisfied:  True
All-aspect morphology recommendation
	-> Input filters:
		Premise satisfied:  True


# Excecute inference

Rule application returns a collection of resources to display in the atlas

By default the premise is checked again when the rule is applied

In [18]:
for rule in morph_rules:
    print(rule["name"])
    print("\t Results:")
    results = apply_rule(allocate_forge_session, rule, input_filters)
    for r in results:
        print("\t\t", r)
    print()

Shape-based morphology recommendation
	 Results:
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/e7634847-3903-4d18-a94e-fc8fb2be3f7c'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/15c8a4c5-807c-4512-bc03-15b7ff987831'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/2d25225f-cdf5-4c55-9f3c-b4f4c71ed1e5'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/dbb4a43e-ff26-410b-9408-eab18005f06f'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/a1983469-d70a-4ff9-9d85-dad984b6b930'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/d4d82091-9e50-47d5-97b8-3609578cfca7'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/d96779ca-fa70-490b-af1a-de1f8cc21d3b'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/9959c647-8942-445d-b2f6-4acb511d62c9'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/n

## Real atlas example

In [19]:
brain_regions = [
    # non cortical
    "mba:549", "mba:856", "mba:864", "mba:239", "mba:958", "mba:1014",
    # cortical
    "mba:315", "mba:541", "mba:97"
]

In [20]:
mtypes = ["http://uri.interlex.org/base/ilx_0381377", "http://uri.interlex.org/base/ilx_038"]

In [21]:
rule = rules_forge.as_json(rules_forge.retrieve(
    "https://bbp.epfl.ch/neurosciencegraph/data/99b71e48-abd5-49e9-965e-a08534b5f0fa"))

In [22]:
input_filters = {
    "BrainRegionQueryParameter": brain_regions,
    "MTypeQueryParameter": mtypes
}

In [32]:
satisfies = check_premises(allocate_forge_session, rule, input_filters)
print(" premise satisfied? ", satisfies)
if satisfies:
    results = apply_rule(
        allocate_forge_session, rule, input_filters, premise_check=False)
    print(results)

 premise satisfied?  True
[{'id': 'https://bbp.epfl.ch/neurosciencegraph/data/7b59a92f-309a-4af1-a56b-91766d348c96'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/fc5dc47f-8ed0-4997-a0fc-47ccaeab2186'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/c5761f9a-6880-442f-bd41-657002b49200'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/1bd93ace-dc35-4a33-b2c6-5c2c80adf1f4'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/e45a145f-ff29-4132-80f0-5834b814d4a1'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/906e253c-19f1-4b13-a7ed-7488714bcffb'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/45ef1fcb-7d54-490c-93dd-efd937303a6a'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/7bc108a5-ebaf-43f3-be17-ce3a0b3ccbe0'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/27218823-a510-4579-b7cd-7d85343646fa'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/61a4e5eb-c89c-4e1b-960b-d4fc0fc7d948'}, {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/42ff423a-979